In [1]:
%load_ext autoreload
%autoreload 1
%aimport keyword_match 

import emoji
import re
import random
import pandas as pd 
import numpy as np
from pandarallel import pandarallel
from opencc import OpenCC
from tqdm.auto import tqdm
from zhconv import convert
import copy
import json
from keyword_match import KeyWordExtractor

In [2]:
companys = list(json.load(open("/workspace/ner-starbucks/configs/company_entity.json", 'r', encoding='utf-8')).keys())
print(f"companys: {len(companys)}， {'老乡鸡' in companys}， {'中国' in companys}, {'南京' in companys}")
key_word_extrator = KeyWordExtractor(dictionary=companys, use_fixed=True, case_sensitive=True)


companys: 20955， True， True, False


In [3]:
error_company_entity = json.load(open("/workspace/NER/error_company_entity.json", 'r', encoding='utf-8'))
error_company_entity.__len__()

5556

# 处理得到所有的中国城市名称，用来过滤标注结果中没有城市名称

In [26]:
chinacity = json.load(open("/workspace/NER/data/chinacity.json", 'r', encoding='utf-8'))
pattern = r"'name': '([\u4e00-\u9fff]+)'"
city_set = set()
matches = re.findall(pattern, str(chinacity))
if matches:
    for match in matches:
        # print(match)
        city_set.add(match)
else:
    print("No match found")

In [27]:
city_set.__len__()

2604

# 处理之前ner积累的标注数据

In [4]:
golden_standard_df = pd.read_json("/workspace/NER/bochk/golden_standard 1.json")
print(golden_standard_df.shape)
golden_standard_df = golden_standard_df.reset_index().rename(columns={'index': "docid"})
golden_standard_df.head()


(1000, 2)


,docid,content,labels
0,0,——以捲煙廠為例\n\n\n\n浙江中煙工業有限責任公司 單宇翔 郁鋼 陸海良 高揚...,"[{'label_name': 'Company', 'text_segment': '浙江..."
1,1,原標題：脫歐不明英國樓市受困 中國投資者趁機入場\n\n\n\n本報記者 姚瑤 上海報道\n...,"[{'label_name': 'Location', 'text_segment': '英..."
2,2,原標題：午評：兩市沖高回落賽馬概念午前崛起 高位股現跌停潮\n\n\n\n金融界網站28日訊...,"[{'label_name': 'Source', 'text_segment': '金融界..."
3,3,"原標題：增厚業績扭虧保殼 多家上市公司""賣房過年""\n\n\n\n東方IC/供圖\n\n\n...","[{'label_name': 'Source', 'text_segment': '證券時..."
4,4,"核心閱讀\n\n\n\n隨著相關配套規則開始徵求意見，""科創板""""註冊制""揭開面紗。\n\n...","[{'label_name': 'Organization', 'text_segment'..."


In [5]:
def read_jsonl(path):
    data_list = []
    with open(path, 'r') as f:
        for line in f:
            data_list.append(json.loads(line))
    return data_list

In [6]:
def judge_is_include_entity_this_type(entity_list, entity_type):
    try:
        if isinstance(entity_list, list):
            return any([dic['label_name']==entity_type for dic in entity_list])
        elif isinstance(entity_list, float):
            return False
    except: 
        return False
        

In [7]:
ner_data = pd.DataFrame(read_jsonl("/workspace/NER/bochk/ner_data.jsonl"))
complete_ner_data = pd.concat([ner_data, golden_standard_df])
complete_ner_data['include_company'] = complete_ner_data['labels'].apply(judge_is_include_entity_this_type, args=("Company", ))
complete_ner_data['include_person'] = complete_ner_data['labels'].apply(judge_is_include_entity_this_type, args=("Person", ))


In [8]:
include_target_df = complete_ner_data[(complete_ner_data['include_company']==True) | (complete_ner_data['include_person']==True)]
include_target_df.rename(columns={'labels': "ner"}, inplace=True)
include_target_df.shape, include_target_df.columns

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


((10083, 5),
 Index(['docid', 'content', 'ner', 'include_company', 'include_person'], dtype='object'))

In [9]:
include_target_df['length'] = include_target_df['content'].apply(len)
include_target_df[include_target_df['length']>512].shape

/tmp/ipykernel_58026/3509638205.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  include_target_df['length'] = include_target_df['content'].apply(len)


(7301, 6)

# 处理bochk人工标注的数据

In [10]:
data = pd.read_excel("/workspace/NER/bochk/bochk.xlsx")
data['include_company'] = True
data['include_person'] = True
# data['headline_content'] = data['content']
manual_label_data = pd.concat([data, include_target_df])
manual_label_data.columns

Index(['docid', 'headline', 'content', 'ner', 'matched_keywords',
       'include_company', 'include_person', 'length'],
      dtype='object')

In [11]:
from zhconv import convert
import copy
def split_sentences(text):
    
    sentences_ls = re.split(r'[?!;~。？！；～>\n\r]', text)
    sentences = [sent for sent in sentences_ls if sent.strip() != '']
    # sentences = list(zip(list(range(0, len(sentences))), sentences))
    return sentences

def split_single_sentence(data: pd.DataFrame, sentence_length_limit: int=500):
    """
    讲文章按照句号分割成句子
    :param data:
    :return:
    """
    complete_data_list = []
    for index, row in data.iterrows():
        raw_single_dic = dict(row)
        content = convert(row['content'], 'zh-cn')
        processed_content_ls = split_sentences(content)
        last_sentence_index = 0
        sentence_index = 0
        for i in range(processed_content_ls.__len__()+1):
            # print(last_sentence_index, i)
            text = "。".join(processed_content_ls[last_sentence_index:i+1])

            if text.strip() != "":
                if text.__len__() >= sentence_length_limit:
                    # print("***", last_sentence_index, i, text.__len__(), "。".join(processed_content_ls[last_sentence_index:i]).__len__())
                    raw_single_dic_backup = copy.copy(raw_single_dic)
                    raw_single_dic_backup['content'] = "。".join(processed_content_ls[last_sentence_index:i])
                    raw_single_dic_backup['split_sentence_index'] = sentence_index
                    complete_data_list.append(raw_single_dic_backup)
                    last_sentence_index = i
                    sentence_index += 1
                else:
                    if i+1 == processed_content_ls.__len__():
                        # print("===", last_sentence_index, i, text.__len__())
                        raw_single_dic_backup = copy.copy(raw_single_dic)
                        raw_single_dic_backup['content'] = "。".join(processed_content_ls[last_sentence_index:])
                        raw_single_dic_backup['split_sentence_index'] = sentence_index
                        complete_data_list.append(raw_single_dic_backup)
                        
                    else:
                        pass
    return pd.DataFrame(complete_data_list)

In [12]:
split_sentence_df = split_single_sentence(manual_label_data)
split_sentence_df.shape, split_sentence_df.columns

((42980, 9),
 Index(['docid', 'headline', 'content', 'ner', 'matched_keywords',
        'include_company', 'include_person', 'length', 'split_sentence_index'],
       dtype='object'))

In [13]:
split_sentence_df['length'] = split_sentence_df['content'].apply(len)
split_sentence_df[split_sentence_df['length']> 512].shape

(336, 9)

In [32]:
def get_keyword(row):
    companys = set()
    person = set()
    if isinstance(row['ner'], str):
        ner_list = eval(row['ner'])
    elif isinstance(row['ner'], list):
        ner_list = row['ner']
    # print(ner_list)
    for dic in ner_list:
        entity = convert(dic['text_segment'], 'zh-cn')
        if entity in row['content']:
            if dic['label_name']=='Company':
                companys.add(entity)
            elif dic['label_name']=='Person' :
                person.add(entity) 
                
    content = convert(row['content'], 'zh-cn')
    company_entity = key_word_extrator.extract(content)
    keyword_matched = set([tup[0] for tup in company_entity])
    companys.update(keyword_matched)
    fix_companys = [ent for ent in companys if ent not in error_company_entity and ent not in city_set and not ent.endswith("大学") and ent not in ("汇聚", '自动化', "EDA")]
    return list(fix_companys), list(person)
    

split_sentence_df[['Companys', "Persons"]] = split_sentence_df.apply(get_keyword, axis=1, result_type='expand')

In [33]:
# split_sentence_df.head(3).to_dict('record')
split_sentence_df.head(3)

,docid,headline,content,ner,matched_keywords,include_company,include_person,length,split_sentence_index,Companys,Persons
0,2023063021774796465,NaN,宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。宝申控股(08151) 有关复牌...,"[{'label_name': 'Company', 'text_segment': '寶申...","['寶申控股', '寶申控股']",True,True,74,0,[宝申控股],[]
1,2023063021774796485,NaN,利骏集团香港(08360) 股东特别大会的结果。利骏集团香港(08360) 于二零二三年六月...,"[{'label_name': 'Company', 'text_segment': '利駿...","['利駿集團', '利駿集團香港']",True,True,74,0,"[利骏集团, 利骏集团香港, 东特]",[]
2,2023063021774796605,NaN,推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...,"[{'label_name': 'Company', 'text_segment': '21...","['21世紀經濟報道', '郭晨', '胡廣傑', '陳之常', '東南大學', '黃如',...",True,True,488,0,[21世纪经济报道],"[陈之常, 杨学鹏, 郭晨, 黄如, 徐光辉, 胡广杰, 吴刚]"


In [34]:
print(split_sentence_df['docid'].is_unique, split_sentence_df['docid'].is_unique)
split_sentence_df['docid'] = split_sentence_df.apply(lambda row: str(row["docid"]) + "&&" + str(row['split_sentence_index']), axis=1)
print(split_sentence_df['docid'].is_unique, split_sentence_df['docid'].is_unique)

False False
True True


In [35]:
import pickle
pickle.dump(split_sentence_df, open("/workspace/NER/data/complete_have_ner_labels_df.xlsx", 'wb'))

## clean context

In [36]:
def clean_data(example):
    
    text = emoji.replace_emoji(example, replace="")
    # text = OpenCC('t2s.json').convert(text)
    text = convert(text, 'zh-cn')
    text = ''.join([x for x in text if x.isprintable()])
    text = text.lower()
    return text

pandarallel.initialize(nb_workers=32, progress_bar=True, use_memory_fs=False)
split_sentence_df["context_cleaned"] = split_sentence_df.parallel_apply(lambda x: clean_data(x["content"]), axis=1)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


## keyword list 回扫

In [37]:
split_sentence_df.head()

,docid,headline,content,ner,matched_keywords,include_company,include_person,length,split_sentence_index,Companys,Persons,context_cleaned
0,2023063021774796465&&0,NaN,宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。宝申控股(08151) 有关复牌...,"[{'label_name': 'Company', 'text_segment': '寶申...","['寶申控股', '寶申控股']",True,True,74,0,[宝申控股],[],宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。宝申控股(08151) 有关复牌...
1,2023063021774796485&&0,NaN,利骏集团香港(08360) 股东特别大会的结果。利骏集团香港(08360) 于二零二三年六月...,"[{'label_name': 'Company', 'text_segment': '利駿...","['利駿集團', '利駿集團香港']",True,True,74,0,"[利骏集团, 利骏集团香港, 东特]",[],利骏集团香港(08360) 股东特别大会的结果。利骏集团香港(08360) 于二零二三年六月...
2,2023063021774796605&&0,NaN,推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...,"[{'label_name': 'Company', 'text_segment': '21...","['21世紀經濟報道', '郭晨', '胡廣傑', '陳之常', '東南大學', '黃如',...",True,True,488,0,[21世纪经济报道],"[陈之常, 杨学鹏, 郭晨, 黄如, 徐光辉, 胡广杰, 吴刚]",推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...
3,2023063021774796605&&1,NaN,"江苏省副省长胡广杰对EDA国创中心成立表示祝贺。他说,集成电路产业是我省具有较强竞争力的优势...","[{'label_name': 'Company', 'text_segment': '21...","['21世紀經濟報道', '郭晨', '胡廣傑', '陳之常', '東南大學', '黃如',...",True,True,432,1,[],"[陈之常, 胡广杰]","江苏省副省长胡广杰对eda国创中心成立表示祝贺。他说,集成电路产业是我省具有较强竞争力的优势..."
4,2023063021774796605&&2,NaN,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[{'label_name': 'Company', 'text_segment': '21...","['21世紀經濟報道', '郭晨', '胡廣傑', '陳之常', '東南大學', '黃如',...",True,True,471,2,[],[黄如],"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题..."


In [38]:
def entend_label_and_merge_content_keyword(example, threshold=-1.0):
    brand_labels = [convert(word, 'zh-cn') for word in example['Companys'].copy()]
    product_labels = [convert(word, 'zh-cn') for word in example['Persons'].copy()]
    text = example['context_cleaned']
    # brand_matched_keywords, product_matched_keywords = [], []
    # if pd.notna(text):
    #     for keyword in brand_list:
    #         if keyword.lower() in text.lower():
    #             brand_matched_keywords.append(keyword.lower())
    #     for keyword in product_list:
    #         if keyword.lower() in text.lower():
    #             product_matched_keywords.append(keyword.lower())

    # brand_labels.extend(brand_matched_keywords)
    # product_labels.extend(product_matched_keywords)
    brand_labels = list(set(brand_labels))
    product_labels = list(set(product_labels))
    brand_labels_, product_labels_ = [], []
    for i in brand_labels:
        temp = brand_labels.copy()
        temp.remove(i)
        flag = 1
        for j in temp:
            if i not in j:
                continue
            else:
                flag = 0
                break
        if flag:
            brand_labels_.append(i)
    for i in product_labels:
        temp = product_labels.copy()
        temp.remove(i)
        flag = 1
        for j in temp:
            if i not in j:
                continue
            else:
                flag = 0
                break
        if flag:
            product_labels_.append(i)
    
    labels_ = brand_labels_ + product_labels_
    

    text_list = []
    separator = "。！？#?!"
    start, end = 0, 0
    while end < len(text):
        if text[end] in separator:
            text_list.append(text[start:end+1])
            start, end = end + 1, end + 1
        else:
            end += 1
        
        if end == len(text) - 1:
            text_list.append(text[start:end+1])

    for idx, t in enumerate(text_list):
        temp = []
        for l in labels_:
            if l in t:
                temp.append(l)
        if temp != []:
            text_list[idx] = text_list[idx] + '||可能的实体：' + ",".join(temp) + '||' 
    return product_labels_, brand_labels_, labels_, "".join(text_list)

In [39]:
split_sentence_df[["person_matched", "company_matched", "matched_keywords", "context_keywords"]] = split_sentence_df.parallel_apply(entend_label_and_merge_content_keyword, axis=1).to_list()
split_sentence_df

/opt/conda/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,docid,headline,content,ner,matched_keywords,include_company,include_person,length,split_sentence_index,Companys,Persons,context_cleaned,person_matched,company_matched,context_keywords
0,2023063021774796465&&0,NaN,宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。宝申控股(08151) 有关复牌...,"[{'label_name': 'Company', 'text_segment': '寶申...",[宝申控股],True,True,74,0,[宝申控股],[],宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。宝申控股(08151) 有关复牌...,[],[宝申控股],宝申控股(08151) 停牌 / 内幕消息 / 其他-杂项。||可能的实体：宝申控股||宝申...
1,2023063021774796485&&0,NaN,利骏集团香港(08360) 股东特别大会的结果。利骏集团香港(08360) 于二零二三年六月...,"[{'label_name': 'Company', 'text_segment': '利駿...","[利骏集团香港, 东特]",True,True,74,0,"[利骏集团, 利骏集团香港, 东特]",[],利骏集团香港(08360) 股东特别大会的结果。利骏集团香港(08360) 于二零二三年六月...,[],"[利骏集团香港, 东特]","利骏集团香港(08360) 股东特别大会的结果。||可能的实体：利骏集团香港,东特||利骏集..."
2,2023063021774796605&&0,NaN,推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...,"[{'label_name': 'Company', 'text_segment': '21...","[21世纪经济报道, 陈之常, 杨学鹏, 郭晨, 黄如, 徐光辉, 胡广杰, 吴刚]",True,True,488,0,[21世纪经济报道],"[陈之常, 杨学鹏, 郭晨, 黄如, 徐光辉, 胡广杰, 吴刚]",推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...,"[陈之常, 杨学鹏, 郭晨, 黄如, 徐光辉, 胡广杰, 吴刚]",[21世纪经济报道],推进科技自立自强， 国家集成电路设计自动化技术创新中心正式揭牌。21世纪经济报道记者 郭晨 ...
3,2023063021774796605&&1,NaN,"江苏省副省长胡广杰对EDA国创中心成立表示祝贺。他说,集成电路产业是我省具有较强竞争力的优势...","[{'label_name': 'Company', 'text_segment': '21...","[陈之常, 胡广杰]",True,True,432,1,[],"[陈之常, 胡广杰]","江苏省副省长胡广杰对eda国创中心成立表示祝贺。他说,集成电路产业是我省具有较强竞争力的优势...","[陈之常, 胡广杰]",[],"江苏省副省长胡广杰对eda国创中心成立表示祝贺。||可能的实体：胡广杰||他说,集成电路产业..."
4,2023063021774796605&&2,NaN,"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...","[{'label_name': 'Company', 'text_segment': '21...",[黄如],True,True,471,2,[],[黄如],"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题...",[黄如],[],"突出科产融合,推动创新链、产业链、资金链、人才链深度融合,着力解决一批基础理论和技术原理问题..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42975,998&&0,NaN,原标题：多城“人才争夺战” 房地产市场格局难变。“二十一世纪什么最贵。人才。”直至十余年后，...,"[{'label_name': 'Time', 'text_segment': '二十一世紀...",[方米],True,True,489,0,[方米],[],原标题：多城“人才争夺战” 房地产市场格局难变。“二十一世纪什么最贵。人才。”直至十余年后，...,[],[方米],原标题：多城“人才争夺战” 房地产市场格局难变。“二十一世纪什么最贵。人才。”直至十余年后，...
42976,998&&1,NaN,工作期间保障交通工具。无独有偶。博州人才政策对于住房方面的关注在其他三四五线城市动作中也有所...,"[{'label_name': 'Time', 'text_segment': '二十一世紀...","[国际金融报, 交通, 工程, 易居研究院智库中心, 严跃进]",True,True,493,1,"[工程, 国际金融报, 易居研究院智库中心, 易居, 交通, 易居研究院]",[严跃进],工作期间保障交通工具。无独有偶。博州人才政策对于住房方面的关注在其他三四五线城市动作中也有所...,[严跃进],"[国际金融报, 交通, 工程, 易居研究院智库中心]",工作期间保障交通工具。||可能的实体：交通||无独有偶。博州人才政策对于住房方面的关注在其他...
42977,998&&2,NaN,这场对于人才的竞争已不再局限于城市之间。“战火”已悄悄弥漫至区、县等更小的行政单位。不过，一...,"[{'label_name': 'Time', 'text_segment': '二十一世紀...",[国际金融报],True,True,447,2,[国际金融报],[],这场对于人才的竞争已不再局限于城市之间。“战火”已悄悄弥漫至区、县等更小的行政单位。不过，一...,[],[国际金融报],这场对于人才的竞争已不再局限于城市之间。“战火”已悄悄弥漫至区、县等更小的行政单位。不过，一...
42978,998&&3,NaN,3月18日，河北石家庄推动户口迁入“零门槛”，群众仅凭居民身份证、户口簿就可向落户地派出所申...,"[{'label_name': 'Time', 'text_segment': '二十一世紀...","[易居, 方米, 严跃进, 沈路]",True,True,496,3,"[易居, 方米]","[严跃进, 沈路]",3月18日，河北石家庄推动户口迁入“零门槛”，群众仅凭居民身份证、户口簿就可向落户地派出所申...,"[严跃进, 沈路]","[易居, 方米]",3月18日，河北石家庄推动户口迁入“零门槛”，群众仅凭居民身份证、户口簿就可向落户地派出所申...


In [75]:
# split_sentence_df.head().to_dict("record")

In [40]:
split_sentence_df['sign'] = split_sentence_df.apply(lambda row: row['Companys'].__len__() == 0 and row['Persons'].__len__() == 0, axis=1)


In [42]:
split_sentence_df[split_sentence_df['sign']==True].shape, split_sentence_df[split_sentence_df['sign']==False].shape

((3387, 16), (39593, 16))

In [43]:
split_sentence_df[split_sentence_df['sign']==False].to_excel("/workspace/NER/bochk/manual_label_data.xlsx", index=False)